In [3]:
# Import libraries
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [4]:
# import data set

data = pd.read_csv('tested_molecules.csv')

In [5]:
# Generate descriptors for each molecule
descriptor_list = [d[0] for d in Descriptors._descList]
calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_list)


# function to get the descriptor for each molecule in SMILES
def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        return calculator.CalcDescriptors(mol)
    return [None] * len(descriptor_list)

In [6]:
# SMILES contain the molecules, so apply the calculate_descriptors on SMILES
data['Descriptors'] = data['SMILES'].apply(compute_descriptors)
descriptors_df = pd.DataFrame(data['Descriptors'].tolist(), columns=descriptor_list)
data = pd.concat([data, descriptors_df], axis=1)

# Drop rows with NaN values
data.dropna(inplace=True)

In [7]:
# standerdize data with standard scaler
#scaler = StandardScaler()
df_scaled = StandardScaler().fit_transform(descriptors_df)

In [8]:
# Apply PCA 
pca = PCA()
principal_components = pca.fit_transform(df_scaled)

# compute cumalative variance
cumulative_variance = pca.explained_variance_ratio_.cumsum()

# get amount of component to cover 80% of variance
n_components_80_variance = (cumulative_variance >= 0.8).argmax() + 1  # index at 0 so add 1
print(f"Number of components to explain 80% variance: {n_components_80_variance}")

Number of components to explain 80% variance: 35


In [9]:
#get loadings
loadings = pca.components_

In [10]:
# make loading matrix
features = descriptors_df.columns
loading_matrix = pd.DataFrame(loadings, columns=features, index=[f'PC{i+1}' for i in range(len(loadings))])
print(loading_matrix)

       MaxEStateIndex  MinEStateIndex  MaxAbsEStateIndex  MinAbsEStateIndex  \
PC1          0.074857   -4.538539e-02       7.485724e-02      -5.772916e-02   
PC2          0.099438   -9.163471e-02       9.943846e-02      -8.823757e-02   
PC3         -0.098613    1.314266e-01      -9.861322e-02       7.659648e-02   
PC4          0.033748   -1.116525e-01       3.374828e-02      -1.980827e-02   
PC5          0.113003   -1.171973e-01       1.130027e-01      -5.123494e-02   
...               ...             ...                ...                ...   
PC204        0.000000   -1.062465e-19      -1.301744e-21      -2.716215e-20   
PC205        0.000000    1.026257e-18       3.169445e-21       1.564543e-19   
PC206       -0.000000    7.399298e-33       6.168914e-34       6.872828e-34   
PC207       -0.000000    1.121632e-34       5.756544e-34      -4.448630e-35   
PC208        0.707107    2.969847e-15      -7.071068e-01      -4.996004e-16   

                qed         MolWt  HeavyAtomMolWt  

In [11]:
# get absolute loading and print it
absolute_loadings = np.abs(loading_matrix).sum(axis=0).sort_values(ascending=False)
print("Sum of Absolute Loadings (Feature Importance):\n", absolute_loadings)

Sum of Absolute Loadings (Feature Importance):
 SMR_VSA10            10.818865
SlogP_VSA8           10.726949
SMR_VSA5             10.614510
SlogP_VSA1           10.517149
SlogP_VSA5           10.429701
                       ...    
fr_phos_acid          1.431506
fr_isothiocyan        1.411940
fr_term_acetylene     1.000641
fr_thiocyan           1.000641
fr_isocyan            1.000011
Length: 208, dtype: float64


In [14]:
# threshold in %
threshold = 10
# get significant features
significant_features = absolute_loadings[absolute_loadings > threshold].index.tolist()
print(f"There are {len(significant_features)} significant features based on threshold: {significant_features}" )

There are 25 significant features based on threshold: ['SMR_VSA10', 'SlogP_VSA8', 'SMR_VSA5', 'SlogP_VSA1', 'SlogP_VSA5', 'PEOE_VSA12', 'EState_VSA1', 'SMR_VSA6', 'EState_VSA7', 'EState_VSA3', 'PEOE_VSA9', 'SMR_VSA7', 'SlogP_VSA3', 'SMR_VSA9', 'PEOE_VSA7', 'SlogP_VSA11', 'PEOE_VSA6', 'EState_VSA8', 'EState_VSA6', 'PEOE_VSA3', 'SMR_VSA4', 'SlogP_VSA2', 'EState_VSA2', 'SlogP_VSA6', 'PEOE_VSA11']
